In [ ]:
"""Generating holdings

Generating holdings with the movements engine

Attributes
----------
cocoon
instruments
transactions
transaction configuration
"""

## Generating holdings with the movements engine

In this notebook we will show you how to <b>create</b> and <b>modify</b> holdings in your portfolio using LUSID's movements engine:

1. Create holdings by booking `Transactions`

2. Create holdings by setting `Holdings` directly

3. Create custom `Transaction Types` and `Sub-holding Keys` to control how LUSID's movements engine builds `Holdings`

To demonstrate, consider the example of a simple equity portfolio holding FTSE100 stocks. First, we'll book transactions to generate holdings. Secondly, we'll set the holdings directly. Finally, we will create a custom transaction type to show how the holdings can be modified. For that example, we'll create a new `Transaction Type` to represent a stock lending transaction and show how this new `Transaction Type` impacts the holding report using `Sub-holding Keys`. 

<br>


> **Key message:**  LUSID's movements engine provides you with flexibility to group, pivot and organise your holdings data.


### 0.1) Import Python packages (LUSID and others)

In [1]:
# Import the general purpose Python packages

import copy
import os
import pytz
from datetime import datetime
import warnings

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)
from lusidtools.cocoon.transaction_type_upload import (
    create_transaction_type_configuration,
)

import lusid
import lusid.models as models
import globalfund as global_fund_tools

# Import data wrangling packages
# One we havent used in other notebooks is the pandas_utils package
# We use this to flatten the results of a get_holdings response to parse the data into a neat DataFrame

from IPython.display import display_html
import pandas as pd
import json

pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

### 0.2) Setup and load data

Define two scopes and a portfolio:

* The first <b>transactions_scope</b> will be used to show a holdings created by Transactions
* The second <b>holdings_scope</b> will be used to show holdings created by setting holdings

In [2]:
# define a scope and a porfolio to to used in this notebook
transactions_scope = "GenerateWithUpsertTransactions"
holdings_scope = "GenerateWithSetHoldings"

# define a portfolio code 
portfolio_code = "EQUITY_UK"

# define some dates we'll use below
after_loan_date = "2020-02-20"
before_loan_date = "2020-01-31"

Load a mapping file to format our DataFrame columns:

In [3]:
# Load a file to format holding response
with open(r"config/generate-holdings/get_holdings_mapping.json") as mappings_file:
    format_holdings_response = json.load(mappings_file)

Define the transactions portfolio API

In [4]:
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

Define some useful functions for use below

In [5]:
def get_holdings_df(scope, code, date=datetime.now(pytz.UTC)):

    holdings_response = transaction_portfolios_api.get_holdings(
        scope=scope,
        code=code,
        property_keys=["Instrument/default/Name"],
        effective_at=date,
    )

    holdings_df = lusid_response_to_data_frame(
        holdings_response,
        rename_properties=True,
        column_name_mapping=format_holdings_response,
    )

    return holdings_df

### 0.3) Read in data from external files

In [6]:
# Load in transactions CSV file

transactions_df = pd.read_csv(
    "data/generate-holdings/generate_holdings_transactions.csv"
)
transactions_df["StockLendingStatus"] = "PortfolioPosition"
transactions_df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions,StockLendingStatus
0,EQUITY_UK,Generic UK Equity portfolio,GBP,AVI,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,02/01/2020,04/01/2020,120000,5,600000,GBP,ftse_tracker,NaN,PortfolioPosition
1,EQUITY_UK,Generic UK Equity portfolio,GBP,AVI,SEDOL1,equity,EQ_1234,Aviva,trd_0002,Buy,15/01/2020,17/01/2020,12000,5,60000,GBP,ftse_tracker,NaN,PortfolioPosition
2,EQUITY_UK,Generic UK Equity portfolio,GBP,BH,SEDOL2,equity,EQ_1235,BHP,trd_0003,Buy,02/01/2020,04/01/2020,60000,18,1080000,GBP,ftse_tracker,NaN,PortfolioPosition
3,EQUITY_UK,Generic UK Equity portfolio,GBP,BH,SEDOL2,equity,EQ_1235,BHP,trd_0004,Buy,15/01/2020,17/01/2020,60000,18,1080000,GBP,ftse_tracker,NaN,PortfolioPosition
4,EQUITY_UK,Generic UK Equity portfolio,GBP,BARC,SEDOL3,equity,EQ_1236,Barclays,trd_0005,Buy,02/01/2020,04/01/2020,150000,2,300000,GBP,ftse_tracker,NaN,PortfolioPosition
5,EQUITY_UK,Generic UK Equity portfolio,GBP,BARC,SEDOL3,equity,EQ_1236,Barclays,trd_0006,Buy,15/01/2020,17/01/2020,150000,2,300000,GBP,ftse_tracker,NaN,PortfolioPosition
6,EQUITY_UK,Generic UK Equity portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,cash_001,FundsIn,02/01/2020,04/01/2020,12000000,1,5000000,GBP,ftse_tracker,GBP,PortfolioPosition


In [7]:
# Load in holdings CSV file

holdings_df = pd.read_csv("data/generate-holdings/set_holdings.csv")
holdings_df

,portfolio_code,instrument_name,quantity,price,currency,ClientInternal,cash_transactions
0,EQUITY_UK,Aviva,132000,5,GBP,EQ_1234,NaN
1,EQUITY_UK,BHP,120000,18,GBP,EQ_1235,NaN
2,EQUITY_UK,Barclays,300000,2,GBP,EQ_1236,NaN
3,EQUITY_UK,Cash,8580000,1,GBP,GBP,GBP


### 0.4) Setup the instrument master 

Here we setup an instrument master. 

In [8]:
instruments_df = transactions_df[["instrument_id", "name"]]

instrument_mapping = {
    "identifier_mapping": {"ClientInternal": "instrument_id"},
    "required": {"name": "name"},
    "properties": [],
}

In [9]:
upsert_instrument_response = load_from_data_frame(
    api_factory=api_factory,
    scope=transactions_scope,
    data_frame=instruments_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
    property_columns=instrument_mapping["properties"],
    properties_scope=transactions_scope,
)

succ, failed, errors = format_instruments_response(upsert_instrument_response)
display(
    pd.DataFrame(
        data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
    )
)

,success,failed,errors
0,4,0,0


### 0.5) Setup the portfolio 

Next we setup a portfolio called EQUITY_UK in both scopes.

In [10]:
portfolio_df = transactions_df[
    ["portfolio_code", "portfolio_name", "portfolio_base_currency"]
]
portfolio_df.head(1)

,portfolio_code,portfolio_name,portfolio_base_currency
0,EQUITY_UK,Generic UK Equity portfolio,GBP


In [11]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "portfolio_base_currency",
        "created": "$2018-01-01T00:00:00+00:00",
    }
}

In [12]:
for scope in transactions_scope, holdings_scope:

    upsert_portfolio_scope = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=portfolio_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional={},
        file_type="portfolios",
        sub_holding_keys=["Transaction/GenerateHoldings/StockLendingStatus"],
    )

    succ, failed = format_portfolios_response(upsert_portfolio_scope)
    display(
        pd.DataFrame(
            data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
        )
    )

,success,failed,errors
0,1,0,0


,success,failed,errors
0,1,0,0


### 1.1) Post transactions to generate holdings

First we post some transactions into EQUITY_UK in the <b>transactions_scope</b>.

In [13]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Currency": "cash_transactions",
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": ["StockLendingStatus"],
}

In [14]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=transactions_scope,
    data_frame=transactions_df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope="GenerateHoldings",
)

succ, failed = format_transactions_response(result)
display(
    pd.DataFrame(
        data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
    )
)

,success,failed,errors
0,1,0,0


Check the holdings.

We see 3 equity positions and 1 GBP cash line.

In [15]:
get_holdings_df(transactions_scope, portfolio_code, date=before_loan_date)

,LusidInstrumentId,StockLendingStatus(GenerateHoldings-SubHoldingKeys),InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
0,LUID_ATFGUBHS,PortfolioPosition,Aviva,EQUITY_UK,GenerateWithUpsertTransactions,P,132000.00,132000.00,660000.00,GBP,0.00,GBP
1,LUID_7XM08GZF,PortfolioPosition,BHP,EQUITY_UK,GenerateWithUpsertTransactions,P,120000.00,120000.00,2160000.00,GBP,0.00,GBP
2,LUID_STGB38I6,PortfolioPosition,Barclays,EQUITY_UK,GenerateWithUpsertTransactions,P,300000.00,300000.00,600000.00,GBP,0.00,GBP
3,CCY_GBP,PortfolioPosition,CCY_GBP,EQUITY_UK,GenerateWithUpsertTransactions,B,8580000.00,8580000.00,8580000.00,GBP,0.00,GBP


### 2.1) Set holdings using holdings file

Secondly, we set holdings directly using SetHoldings.

In [16]:
holdings_df["market_value"] = holdings_df["price"] * holdings_df["quantity"]
holdings_df["StockLendingStatus"] = "PortfolioPosition"
holdings_df

,portfolio_code,instrument_name,quantity,price,currency,ClientInternal,cash_transactions,market_value,StockLendingStatus
0,EQUITY_UK,Aviva,132000,5,GBP,EQ_1234,NaN,660000,PortfolioPosition
1,EQUITY_UK,BHP,120000,18,GBP,EQ_1235,NaN,2160000,PortfolioPosition
2,EQUITY_UK,Barclays,300000,2,GBP,EQ_1236,NaN,600000,PortfolioPosition
3,EQUITY_UK,Cash,8580000,1,GBP,GBP,GBP,8580000,PortfolioPosition


In [17]:
holdings_mapping = {
    "name": "instrument_name",
    "effective_at": "$2020-01-01",
    "code": "portfolio_code",
    "tax_lots.units": "quantity",
    "tax_lots.price": "price",
    "tax_lots.portfolio_cost": "market_value",
    "tax_lots.cost.currency": "currency",
    "tax_lots.cost.amount": "market_value",
}

identifiers = {"ClientInternal": "ClientInternal", "Currency": "cash_transactions"}

In [18]:
holdings_response = load_from_data_frame(
    api_factory=api_factory,
    scope=holdings_scope,
    data_frame=holdings_df,
    mapping_required=holdings_mapping,
    mapping_optional={},
    file_type="holdings",
    identifier_mapping=identifiers,
    sub_holding_keys=["StockLendingStatus"],
)

Again, we see three equity positions and one GBP cash line.

In [19]:
get_holdings_df(holdings_scope, portfolio_code, date=before_loan_date)

,LusidInstrumentId,StockLendingStatus(GenerateHoldings-SubHoldingKeys),InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
0,LUID_ATFGUBHS,<Not Classified>,Aviva,EQUITY_UK,GenerateWithSetHoldings,P,132000.00,132000.00,660000.00,GBP,660000.00,GBP
1,LUID_7XM08GZF,<Not Classified>,BHP,EQUITY_UK,GenerateWithSetHoldings,P,120000.00,120000.00,2160000.00,GBP,2160000.00,GBP
2,LUID_STGB38I6,<Not Classified>,Barclays,EQUITY_UK,GenerateWithSetHoldings,P,300000.00,300000.00,600000.00,GBP,600000.00,GBP
3,CCY_GBP,<Not Classified>,CCY_GBP,EQUITY_UK,GenerateWithSetHoldings,B,8580000.00,8580000.00,8580000.00,GBP,8580000.00,GBP


### 3.1) Configure a new transaction type for a stock lending transaction

In this section, we create a new `transaction type` to capture a stock lending transaction. When we loan a stock, we want to capture the loan amount in the IBOR, but it should be segragated from the other non-loan positions.  

In [20]:
# Configure a new transaction type

movement = [
    models.TransactionConfigurationMovementDataRequest(
        movement_types="StockMovement",
        side="Side1",
        direction=-1,
        properties={},
        mappings=[
            models.TransactionPropertyMappingRequest(
                property_key=f"Transaction/GenerateHoldings/StockLendingStatus",
                set_to="PortfolioPosition",
            )
        ],
    ),
    models.TransactionConfigurationMovementDataRequest(
        movement_types="StockMovement",
        side="Side1",
        direction=1,
        properties={},
        mappings=[
            models.TransactionPropertyMappingRequest(
                property_key=f"Transaction/GenerateHoldings/StockLendingStatus",
                set_to="Loan",
            )
        ],
    ),
]


alias = models.TransactionConfigurationTypeAlias(
    type="StockLoanOut",
    description="Booking of stock lending transaction out",
    transaction_class="StockLending",
    transaction_group="default",
    transaction_roles="Shorter",
)
response = create_transaction_type_configuration(api_factory, alias, movement)

### 3.2) Post the stock lending transaction into our portfolio 

In [21]:
stocklend_df = pd.read_csv("data/generate-holdings/stock_lend.csv")
stocklend_df["txn_type"] = "StockLoanOut"
stocklend_df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK,Generic UK Equity portfolio,GBP,BARC,SEDOL3,equity,EQ_1236,Barclays,stocklend_0001,StockLoanOut,15/02/2020,17/02/2020,150000,2,300000,GBP,ftse_tracker,nan


In [22]:
for scope in transactions_scope, holdings_scope:

    stock_lending_transaction = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=stocklend_df,
        mapping_required=transaction_mapping["required"],
        mapping_optional=transaction_mapping["optional"],
        file_type="transactions",
        identifier_mapping=transaction_mapping["identifier_mapping"],
    )

succ, failed = format_transactions_response(result)
display(
    pd.DataFrame(
        data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
    )
)

,success,failed,errors
0,1,0,0


### 3.3) Check holdings after the stock lending transaction

Result - we can see the <i>Barclays PLC</i> holding has been split in a <b>Loan</b> and <b>PortfolioPosition</b> amount. 

In [23]:
get_holdings_df(transactions_scope, portfolio_code, date = after_loan_date)

,LusidInstrumentId,StockLendingStatus(GenerateHoldings-SubHoldingKeys),InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
0,LUID_ATFGUBHS,PortfolioPosition,Aviva,EQUITY_UK,GenerateWithUpsertTransactions,P,132000.00,132000.00,660000.00,GBP,0.00,GBP
1,LUID_7XM08GZF,PortfolioPosition,BHP,EQUITY_UK,GenerateWithUpsertTransactions,P,120000.00,120000.00,2160000.00,GBP,0.00,GBP
2,LUID_STGB38I6,PortfolioPosition,Barclays,EQUITY_UK,GenerateWithUpsertTransactions,P,150000.00,150000.00,300000.00,GBP,0.00,GBP
3,LUID_STGB38I6,Loan,Barclays,EQUITY_UK,GenerateWithUpsertTransactions,P,150000.00,150000.00,300000.00,GBP,0.00,GBP
4,CCY_GBP,PortfolioPosition,CCY_GBP,EQUITY_UK,GenerateWithUpsertTransactions,B,8580000.00,8580000.00,8580000.00,GBP,0.00,GBP
